In [1]:
import gym
import torch
import numpy as np
import sys
from torchvision.transforms import ToTensor, Lambda
from torch import nn
from my_model import Network, policy
from utils import dataset, dataloader, buffer
import trainer
import random
import torch.onnx as onnx
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('RLresult/hopper/dqn')
device = 'cuda' if torch.cuda.is_available() else 'cpu'


/home/jongsu/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
env = gym.make('Hopper-v3') 
print(env.action_space)
print(env.observation_space)

Box([-1. -1. -1.], [1. 1. 1.], (3,), float32)
Box([-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf], (11,), float64)


In [3]:
HIDDEN_SIZE = 32
BATCH_SIZE = 10000
learning_rate = 0.01
HOP_ACTION = 125
CAPACITY = 10000
TRAIN_ITER = CAPACITY*10/BATCH_SIZE
PARAM_PATH = 'DQN_param/hopperDQN.pth'
PARAM_PATH_TEST = 'DQN_param/hopperDQN_test.pth'
#print(Transition(2,1,2,3))
#Transition(state=2, action=1, next_state=2, reward=3)

In [4]:
input_state_size = len(env.observation_space.sample())
updatedDQN = Network.SimpleNN(input_state_size, HIDDEN_SIZE, HOP_ACTION).to(device)
baseDQN = Network.SimpleNN(input_state_size, HIDDEN_SIZE, HOP_ACTION).to(device)

In [5]:
updatedDQN.load_state_dict(torch.load(PARAM_PATH))
baseDQN.load_state_dict(updatedDQN.state_dict())
baseDQN.eval()

SimpleNN(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=11, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=125, bias=True)
    (5): ReLU()
  )
)

In [6]:
DQN_policy = policy.Policy("DQN", baseDQN, "hope")

In [7]:

mybuffer = buffer.Simulate(env, DQN_policy)
myhopdata = dataset.SimData(capacity = CAPACITY)
mybuffer.renewal_memory(CAPACITY, myhopdata)

In [8]:


mydataloader = dataloader.CustomDataLoader(myhopdata, batch_size=BATCH_SIZE)
mytrain = trainer.Train("hope", mydataloader, "DQN")

In [9]:
optimizer = torch.optim.SGD(updatedDQN.parameters(), lr=learning_rate)

In [10]:
mybuffer.renewal_memory(CAPACITY, myhopdata)

In [10]:
%%time
train_iter = 100
i = 0
#for p in updatedDQN.parameters(): 
#    print(p)
optimizer = torch.optim.SGD(updatedDQN.parameters(), lr=learning_rate)
while(i < train_iter):
    i = i + 1
    mybuffer.renewal_memory(CAPACITY, myhopdata)
    loss = mytrain.training(TRAIN_ITER, BATCH_SIZE, optimizer)
    writer.add_scalar("loss", loss, i)
    torch.save(updatedDQN.state_dict(), PARAM_PATH)
    baseDQN.load_state_dict(updatedDQN.state_dict())
    baseDQN.eval()

env.close()


loss =  tensor(23951.7402, device='cuda:0', grad_fn=<MseLossBackward>)


KeyboardInterrupt: 

In [ ]:
writer.flush()
writer.close()

In [ ]:
'''
print(env.unwrapped.sim.model.get_joint_qpos_addr('root'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('hip_1'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('hip_2'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('hip_3'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('hip_4'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_1'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_2'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_3'))
print(env.unwrapped.sim.model.get_joint_qpos_addr('ankle_4'))
#if u enter <body> in addr then this will show you available list
#('root', 'hip_1', 'ankle_1', 'hip_2', 'ankle_2', 'hip_3', 'ankle_3', 'hip_4', 'ankle_4')
#qpos is front value index
'''
